# Imports

In [1]:
import pandas as pd
import numpy as np
import os
import datetime

# 1. Merge incomplete runs

In [2]:
csv_path1_1 = '../experiments/wireshark/unmuted/test1-rest 7 Runs-whatsapp-2-wireshark.csv'
csv_path1_2 = '../experiments/wireshark/unmuted/test1-Run1-whatsapp-2-wireshark.csv'
output1 = '../experiments/wireshark/unmuted/test1-whatsapp-2-wireshark.csv'
timestamp1 = '2021.10.27_155606'

# csv_path2_1 = 
# csv_path2_2 = 
# output1 = 

to_merge = [(csv_path1_1, csv_path1_2, output1),]

for (csv1, csv2, out) in to_merge:
    df1 = pd.read_csv(csv_path1_1)
    df2 = pd.read_csv(csv_path1_2)

    df2['No.'] = df2['No.'] + df1['No.'][len(df1)-1]

    frames = [df1, df2]

    result = pd.concat(frames,ignore_index=True)

    result.to_csv(output1)


In [29]:
# Participants, Microphone
def add_treatment(df):
    df['Treatment'] = np.nan
    
    df['Treatment'][(df['Participants'] == '2') & (df['Microphone'] == 'muted')] = 'M2'
    df['Treatment'][(df['Participants'] == '2') & (df['Microphone'] == 'unmuted')] = 'U2'
    df['Treatment'][(df['Participants'] == '5') & (df['Microphone'] == 'muted')] = 'M5'
    df['Treatment'][(df['Participants'] == '5') & (df['Microphone'] == 'unmuted')] = 'U5'
    
    df = df[['App', 'Treatment'] + list(df.columns[1:-1])] 
    return df

# new_wireshark_result =  add_treatment(wireshark_result)
# new_trepn_result =  add_treatment(trepn_result)

# new_wireshark_result
# new_trepn_result

# 1. Preprocess csv

In [37]:
pd.options.mode.chained_assignment = None

experiment_folder = '../experiments'
output_folder = '../experiments/processed'

app_list = ['whatsapp', 'instagram', 'facebook']
app_folder = {'whatsapp': 'com-whatsapp', 'facebook': 'com-facebook-orca', 'instagram' : 'com-instagram-android'}
repetitions = 8

wireshark_result = None
trepn_result = None

for profiler in os.listdir(experiment_folder):
    
    if profiler == 'trepn':
        continue

    if profiler == 'wireshark':

        for folder in os.listdir(f'{experiment_folder}/{profiler}'):
            for filename in sorted(os.listdir(f'{experiment_folder}/{profiler}/{folder}')):
                
                if filename.endswith('.csv') and filename.split('-')[1] in app_list:
                    
                    ########### Wireshark ###########
                    
                    print(f"Processing '{folder}/{filename}'...")

                    # read wireshark file
                    df1 = pd.read_csv(f'{experiment_folder}/{profiler}/{folder}/{filename}')

                    df1 = df1[['Time', 'Length']]
                    
                    # add relevant columns
                    df1['App'] = filename.split('-')[1]
                    df1['Participants'] = filename.split('-')[2]
                    df1['Microphone'] = folder
                    df1[['Date', 'Time', 'Miliseconds']] = df1['Time'].str.split(" |,", expand=True) # format time data
                    df1['Avg_packets'] = round(len(df1) / repetitions) # packets are gathered over n repetition
                    
                    # drop non relevant columns
                    df1 = df1.drop(columns=['Miliseconds'])

                    # add to wireshark dataframe
                    frames = [wireshark_result, df1]
                    wireshark_result = pd.concat(frames, ignore_index=True)

                    
                    ########### Combine with Trepn ###########
                    timestamp_file = f"{'-'.join(filename.split('-')[:-1] + ['timestamp'])}.txt"

                    with open(f'{experiment_folder}/{profiler}/{folder}/{timestamp_file}') as f:
                        lines = f.readlines()
                        timestamps = []
                        
                        for i,line in enumerate(lines):
                            if 'Logs Directory =' in line and 'Aggregated' in lines[i-1]:
                                timestamps = [line.split('/')[-5]]
                                
                            elif '/2021.' in line and '2021' in line.split('/')[-5]:
                                timestamps = [line.split('/')[-5]]

                        if filename in output1:
                            timestamps.append(timestamp1)
#                             print('- ***Warning: this file must have multiple timestamp/trepn files due to a failed run!***')
                                
                                
                        assert timestamps != [], 'Oh oh, no timestamp file found!'            
                    
                    for timestamp in timestamps:
                        # determine trepn folder
                        app = filename.split('-')[1]
                        trepn_data_folder = f'{experiment_folder}/trepn/{timestamp}/data/nexusWired/{app_folder[app]}/trepn'

                        print(f"- Adding trepn data '{timestamp}'...")

                        for f in sorted(os.listdir(trepn_data_folder)):

                            if f != 'Aggregated.csv':

                                # read trepn file
                                df2 = pd.read_csv(f'{trepn_data_folder}/{f}')

                                # add relevant columns
                                df2['App'] = filename.split('-')[1]
                                df2['Participants'] = filename.split('-')[2]
                                df2['Microphone'] = folder

                                # add to trepn dataframe
                                frames = [trepn_result, df2]
                                trepn_result = pd.concat(frames, ignore_index=True)

                            else:
                                continue

                    print()
                else:
                    continue



                

    #                 resuls.to_csv(f'{output_folder}/p_{filename}')
    
    
# restructure wireshark columns
# wireshark_result = wireshark_result[['Date', 'Time', 'App', 'Participants', 'Microphone', 'Length', 'Avg_packets']]
wireshark_result = wireshark_result[['App', 'Participants', 'Microphone', 'Length', 'Avg_packets']]
wireshark_result = wireshark_result.rename(columns = {'Length':'Bytes'})
wireshark_result =  add_treatment(wireshark_result)
wireshark_result = wireshark_result.rename(columns = {
    'App' : 'app',
    'Treatment': 'treatment',
    'Participants': 'participants',
    'Microphone': 'microphone',
    'Bytes': 'packet size',
    'Avg_packets': 'avg packets'
})


# trepn_result = trepn_result[['App', 'Participants', 'Microphone'] + list(trepn_result.columns[:9])]
trepn_result = trepn_result[['App', 'Participants', 'Microphone'] + list(trepn_result.columns[:9])]
trepn_result =  add_treatment(trepn_result)
trepn_result = trepn_result.rename(columns = {
    'App' : 'app',
    'Treatment': 'treatment',
    'Participants': 'participants',
    'Microphone': 'microphone',
    'Time  [ms]':'ms1',
    'Memory Usage [KB]': 'memory',
    'Time  [ms].1': 'ms2',
    'Battery Power [uW] (Raw)': 'power raw',
    'Battery Power [uW] (Delta)' : 'power delta',
    'Time  [ms].2' : 'ms3',
    'CPU Load [%]' : 'cpu',
    'Time  [ms].3' : 'ms4',
    'CPU Load (Normalized) [%]' : 'cpu norm'
    })
trepn_result = trepn_result.drop(columns=['cpu', 'power raw', 'ms3'])

# fill nan values with mean values
trepn_result = trepn_result.fillna(trepn_result.mean())
wireshark_result = wireshark_result.fillna(wireshark_result.mean())

# remove nan rows
trepn_result = trepn_result.dropna()
wireshark_result = wireshark_result.dropna()




########### Save processed files ###########
wireshark_result.to_csv(f'{output_folder}/wireshark.csv', index=False)
trepn_result.to_csv(f'{output_folder}/trepn.csv', index=False)

Processing 'muted/test2-facebook-2-wireshark.csv'...
- Adding trepn data '2021.10.27_181054'...

Processing 'muted/test4-facebook-5-wireshark.csv'...
- Adding trepn data '2021.10.27_214616'...

Processing 'muted/test5-whatsapp-5-wireshark.csv'...
- Adding trepn data '2021.10.27_225054'...

Processing 'muted/test6-instagram-2-wireshark.csv'...
- Adding trepn data '2021.10.28_144951'...

Processing 'muted/test8-whatsapp-2-wireshark.csv'...
- Adding trepn data '2021.10.28_174741'...

Processing 'muted/test9-instagram-5-wireshark.csv'...
- Adding trepn data '2021.10.28_180056'...

Processing 'unmuted/test1-whatsapp-2-wireshark.csv'...
- Adding trepn data '2021.10.27_172205'...
- Adding trepn data '2021.10.27_155606'...

Processing 'unmuted/test10-facebook-2-wireshark.csv'...
- Adding trepn data '2021.10.28_190423'...

Processing 'unmuted/test11-whatsapp-5-wireshark.csv'...
- Adding trepn data '2021.10.28_205653'...

Processing 'unmuted/test3-instagram-5-wireshark.csv'...
- Adding trepn dat

In [38]:
wireshark_result

,app,treatment,participants,microphone,packet size,avg packets
0,facebook,M2,2,muted,55,703
1,facebook,M2,2,muted,66,703
2,facebook,M2,2,muted,110,703
3,facebook,M2,2,muted,110,703
4,facebook,M2,2,muted,74,703
...,...,...,...,...,...,...
77320,facebook,U5,5,unmuted,54,710
77321,facebook,U5,5,unmuted,56,710
77322,facebook,U5,5,unmuted,54,710
77323,facebook,U5,5,unmuted,56,710


In [39]:
trepn_result

,app,treatment,participants,microphone,ms1,memory,ms2,power delta,ms4,cpu norm
0,facebook,M2,2,muted,61.0,2707484.0,73.0,0.0,72.0,48.0
1,facebook,M2,2,muted,159.0,2700128.0,165.0,0.0,165.0,47.0
2,facebook,M2,2,muted,260.0,2703192.0,268.0,0.0,268.0,38.0
3,facebook,M2,2,muted,361.0,2703364.0,371.0,0.0,371.0,36.0
4,facebook,M2,2,muted,461.0,2703488.0,468.0,2989584.0,468.0,34.0
...,...,...,...,...,...,...,...,...,...,...
90292,facebook,U5,5,unmuted,132047.0,2712424.0,132067.0,3643967.0,132066.0,30.0
90293,facebook,U5,5,unmuted,132151.0,2709184.0,132165.0,3643967.0,132164.0,30.0
90294,facebook,U5,5,unmuted,132247.0,2709852.0,132274.0,3643967.0,132272.0,30.0
90295,facebook,U5,5,unmuted,132350.0,2709932.0,132380.0,3643967.0,132379.0,30.0
